In [ ]:
# Take start# and end$ into consideration
import pandas as pd
import re
import math

def read_data():
    df = pd.read_excel('./prep_data.xlsx',header = 0)
    df = df[df.columns[1:]]
    words = df['Word'].to_list()
    for i in range(len(words)):
        words[i] = '#' + words[i] + '$'

    data = []
    for i in range(len(words)):
        for letter in list(words[i]):
            data.append(letter)

    return words,data

def create_bigram(data):
    bigram_list = []
    bigram_cnt = {}
    unigram_cnt = {}
    for i in range(len(data)):
        if i < len(data) - 1 and (i % 7 != 6):
            bigram_list.append( (data[i],data[i + 1]))

            if (data[i], data[i + 1]) in bigram_cnt:
                bigram_cnt[(data[i], data[i + 1])] += 1
            else:
                bigram_cnt[(data[i], data[i + 1])] = 1 
        
        if(data[i] in unigram_cnt):
            unigram_cnt[data[i]] += 1
        else:
            unigram_cnt[data[i]] = 1

    return bigram_list,bigram_cnt, unigram_cnt

def get_bigram_prob(bigram_list, bigram_cnt, unigram_cnt):
    bigram_prob = {}
    for bigram in bigram_list:
        letter1 = bigram[0]
        letter2 = bigram[1]
        bigram_prob[bigram] = (bigram_cnt[bigram]) / (unigram_cnt[letter1])

    return bigram_prob

def get_word_prob(word,bigram_prob):
    letters = list(word)
    word_bigram_list = []
    output_prob = 1

    for i in range(len(letters) - 1):
        word_bigram_list.append((letters[i], letters[i + 1]))

    for i in range(len(word_bigram_list)):
        if word_bigram_list[i] in bigram_prob:
            output_prob *= bigram_prob[word_bigram_list[i]]
        else:
            output_prob *= 0
        
    return output_prob

def output_data(words,bigram_prob):
    df = pd.read_excel('./prep_data.xlsx',header = 0)
    df = df[df.columns[1:]]
    prob_lists = []
    for i in range(len(words)):
        prob_lists.append(get_word_prob(words[i],bigram_prob))
    df['probability'] = pd.DataFrame(prob_lists)
    df.to_excel('./myprep_data.xlsx')

    return

## Main Function
words, data = read_data()
bigram_list, bigram_cnt, unigram_cnt = create_bigram(data)
bigram_prob = get_bigram_prob(bigram_list, bigram_cnt, unigram_cnt)
# output_data(words,bigram_prob)


In [47]:
import pandas as pd
import re

def read_data():
    df = pd.read_excel('./prep_data.xlsx',header = 0)
    df = df[df.columns[1:]]
    words = df['Word'].to_list()
    data = []

    for i in range(len(words)):
        for letter in re.findall('[a-z]',words[i]):
            data.append(letter)

    return words,data

def create_bigram(data):
    bigram_list = []
    bigram_cnt = {}
    unigram_cnt = {}
    for i in range(len(data)):
        if i < len(data) - 1 and (i % 5 != 4):
            bigram_list.append( (data[i],data[i + 1]))

            if (data[i], data[i + 1]) in bigram_cnt:
                bigram_cnt[(data[i], data[i + 1])] += 1
            else:
                bigram_cnt[(data[i], data[i + 1])] = 1 
        
        if(data[i] in unigram_cnt):
            unigram_cnt[data[i]] += 1
        else:
            unigram_cnt[data[i]] = 1

    return bigram_list,bigram_cnt, unigram_cnt

def get_bigram_prob(bigram_list, bigram_cnt, unigram_cnt):
    prob_list = {}
    for bigram in bigram_list:
        letter1 = bigram[0]
        letter2 = bigram[1]
        prob_list[bigram] = (bigram_cnt.get(bigram)) / (unigram_cnt.get(letter1))

    return prob_list

## Helper fucntion
def get_word_bigram(word):
    letters = list(word)
    word_bigram_list = []
    for i in range(len(letters)):
        if i < len(letters) - 1:
            word_bigram_list.append((letters[i], letters[i + 1]))
    return word_bigram_list

def get_word_prob(word,bigram_prob):
    word_bigram_list = get_word_bigram(word)
    output_prob = 1

    for i in range(len(word_bigram_list)):
        if word_bigram_list[i] in bigram_prob:
            output_prob *= bigram_prob[word_bigram_list[i]]
        else:
            output_prob *= 0
    return output_prob

def get_vowel_cnt(words):
    vowels = ['a','e','i','o','u']
    vowel_cnt = []
    consont_cnt = []
    for i in range(len(words)):
        vowel_cnt.append(0)
        consont_cnt.append(0)
    
    for i in range(len(words)):
        for letter in list(words[i]):
            if letter in vowels:
                vowel_cnt[i] += 1
            else:
                consont_cnt[i] += 1

    return vowel_cnt

def get_unigram_freq(words, unigram_cnt):

    letters_sum = 0
    for val in unigram_cnt.values():
        letters_sum += val

    unigram_freq = []
    for i in range(len(words)):
        unigram_freq.append(0)
        for letter in list(words[i]):
            unigram_freq[i] += unigram_cnt[letter]

    return unigram_freq

def get_bigram_freq(words, bigram_cnt):
    bigram_freq = [[],[],[],[]]

    for word in words:
        word_bigram_list = get_word_bigram(word)
        for i in range(len(word_bigram_list)):
            freq = bigram_cnt[word_bigram_list[i]]
            bigram_freq[i].append(freq)

    return bigram_freq
## Main Function
words, data = read_data()
bigram_list, bigram_cnt, unigram_cnt = create_bigram(data)

bigram_prob = get_bigram_prob(bigram_list, bigram_cnt, unigram_cnt)


# Get the data
prob_list = []
for word in words:
    prob_list.append(get_word_prob(word,bigram_prob))

vowel_cnt = get_vowel_cnt(words)

unigram_freq = get_unigram_freq(words, unigram_cnt)

bigram_freq = get_bigram_freq(words, bigram_cnt)

In [58]:
# Output Data

df = pd.read_excel('./prep_data.xlsx',header = 0)
df = df[df.columns[1:]]

df['probability'] = pd.DataFrame(prob_list)
df['vowel_num'] = pd.DataFrame(vowel_cnt)
df['letter_freq'] = pd.DataFrame(vowel_cnt)
df['bigram0_freq'] = pd.DataFrame(bigram_freq[0])
df['bigram1_freq'] = pd.DataFrame(bigram_freq[1])
df['bigram2_freq'] = pd.DataFrame(bigram_freq[2])
df['bigram3_freq'] = pd.DataFrame(bigram_freq[3])
df.to_excel('./myprep_data.xlsx')


          Date  Contest_num   Word  Number of  reported results  \
0   2022-01-07          202  slump                        80630   
1   2022-01-08          203  crank                       101503   
2   2022-01-09          204  gorge                        91477   
3   2022-01-10          205  query                       107134   
4   2022-01-11          206  drink                       153880   
..         ...          ...    ...                          ...   
348 2022-12-27          556  condo                        20879   
349 2022-12-28          557  impel                        20160   
350 2022-12-29          558  havoc                        20001   
351 2022-12-30          559  molar                        21204   
352 2022-12-31          560  manly                        20380   

     Number in hard mode     1_try   2_tries    3_tries    4_tries    5_tries  \
0                   1362  1.000000  3.000000  23.000000  39.000000  24.000000   
1                   1763  1.00000